# Overview
This file reads in a list of tickers, gets the correct Eikon name of the ticker, reads in fundamental quarterly data for the list of tickers, and does some intial clean up of the data read in. All data is then stored in csvs.

In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime as dt
import bs4 as bs
import requests
import urllib2

ek.set_app_id('DeNovoQuantFund')

## Stock List
Different functions for pulling stock tickers from different sources.

In [ ]:
# List of S&P 500 tickers

def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    
    return tickers

ticker_list = save_sp500_tickers()

In [ ]:
ticker_list = pd.read_csv('/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NYSE_Ticker_Symbols.csv')

In [ ]:
# Get tickers from NASDAQ text file.
NASDAQ_tickers = open("/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/nasdaqlisted.txt","r")
lines = NASDAQ_tickers.readlines()
ticker_list = []
N = len(lines)
for i, line in enumerate(lines):
    if i > 0 and i < N:
        ticker_list.append(line.split("|")[0])
NASDAQ_tickers.close()

In [ ]:
# Only run for certain files.
ticker_list = np.array(ticker_list['Symbol'],dtype=np.string0)
ticker_list = [tic for tic in ticker_list]

In [ ]:
ticker_list

## Getting relevant fields

List of fields to add:
Fundamental Data
Numerical Data
[
ek.TR_Field('TR.DilutedEpsInclExtra'),
ek.TR_Field('TR.RevenuePerShare'),
ek.TR_Field('TR.BookValuePerShare'),
ek.TR_Field('TR.DPSMean'),
ek.TR_Field('TR.EBITMarginPercent'),
ek.TR_Field('TR.TotalLongTermDebt'),
ek.TR_Field('TR.EBIT'),
ek.TR_Field('TR.PriceClose'),
ek.TR_Field('TR.PriceClose.date')
ek.TR_Field('TR.PE.date'),
ek.TR_Field('TR.DPSActValue'),
ek.TR_Field('TR.DilutedEpsExclExtra'),
ek.TR_Field('TR.TtlDebtToTtlEquityPct'), 
ek.TR_Field('TR.EBITMarginPercent'),
ek.TR_Field('TR.ROATotalAssetsPercent'),
ek.TR_Field('TR.TotalDebtToEV'),
ek.TR_Field('TR.TotalDebtToEBITDA')
ek.TR_Field('TR.OperatingIncome'),
ek.TR_Field('TR.Revenue'),
ek.TR_Field('TR.NormIncAvailToCommon'),
ek.TR_Field('TR.GrossDividendsCmnStock'),
ek.TR_Field('TR.TotalCurrentAssets'),
ek.TR_Field('TR.TotalCurrLiabilities'),
ek.TR_Field('TR.TotalAssetsReported'),
ek.TR_Field('TR.TotalLiabilities'),
ek.TR_Field('TR.TotalLongTermDebt'),
ek.TR_Field('TR.TotalEquity'),
ek.TR_Field('TR.EBIT'),
ek.TR_Field('TR.NetIncomeBeforeTaxes'),
ek.TR_Field('TR.OperatingIncome'),
ek.TR_Field('TR.OperatingExpense'),
ek.TR_Field('TR.CostOfRevenueTotal'),
ek.TR_Field('TR.GrossMargin'),
ek.TR_Field('TR.OperatingMarginPercent'),
ek.TR_Field('TR.TotalDebtOutstanding'),
ek.TR_Field('TR.TotalInventory'),
ek.TR_Field('TR.CashAndSTInvestments'),
ek.TR_Field('TR.TotalReceivablesNet'),
ek.TR_Field('TR.TRBCEconomicSector'),
ek.TR_Field('TR.PropertyPlantEquipmentTotalNet'),
ek.TR_Field('TR.GoodwillNet'),
ek.TR_Field('TR.TangibleBVPS'),
ek.TR_Field('TR.PriceTargetMean'),
ek.TR_Field('TR.NumberOfAnalysts'),
ek.TR_Field('TR.QuickRatio'),
ek.TR_Field('TR.CurrentRatio')
]

Categorical Data
[ek.TR_Field('TR.TRBCEconomicSector'),
ek.TR_Field('TR.CommonName'),
ek.TR_Field('TR.HeadquartersCountry'),
ek.TR_Field('TR.ExchangeName')
]

Technical Daily Data
[
ek.TR_Field('TR.EVToSales'),
ek.TR_Field('TR.PE'),
ek.TR_Field('TR.PriceToSalesPerShare'),
ek.TR_Field('TR.PriceToBVPerShare'),
ek.TR_Field('TR.PriceToCFPerShare'),
ek.TR_Field('TR.TotalDebtToEBITDA'),
ek.TR_Field('TR.TotalDebtToEV'),
ek.TR_Field('TR.PricePctChg1D'),
ek.TR_Field('TR.PricePctChg2D'),
ek.TR_Field('TR.PricePctChg5D'),
ek.TR_Field('TR.PricePctChg4W'),
ek.TR_Field('TR.PricePctChg4M'),
ek.TR_Field('TR.PricePctChg8M'),
ek.TR_Field('TR.PricePctChg11M'),
ek.TR_Field('TR.Volume'),
ek.TR_Field('TR.AvgDailyVolume120D'),
ek.TR_Field('TR.AvgDailyVolume250D'),
ek.TR_Field('TR.PriceAvg100D'),
ek.TR_Field('TR.PriceAvg250D'),

]

In [ ]:
# List of pertinant numerical fields
numerical_fields = [
ek.TR_Field('TR.DilutedEpsInclExtra'),
ek.TR_Field('TR.RevenuePerShare'),
ek.TR_Field('TR.BookValuePerShare'),
ek.TR_Field('TR.DPSMean'),
ek.TR_Field('TR.EBITMarginPercent'),
ek.TR_Field('TR.TotalLongTermDebt'),
ek.TR_Field('TR.EBIT'),
ek.TR_Field('TR.PriceClose'),
ek.TR_Field('TR.PriceClose.date'),
ek.TR_Field('TR.DPSActValue'),
ek.TR_Field('TR.DilutedEpsExclExtra'),
ek.TR_Field('TR.TtlDebtToTtlEquityPct'), 
ek.TR_Field('TR.EBITMarginPercent'),
ek.TR_Field('TR.ROATotalAssetsPercent'),
ek.TR_Field('TR.OperatingIncome'),
ek.TR_Field('TR.TotalRevenue'),
ek.TR_Field('TR.NormIncAvailToCommon'),
ek.TR_Field('TR.GrossDividendsCmnStock'),
ek.TR_Field('TR.TotalCurrentAssets'),
ek.TR_Field('TR.TotalCurrLiabilities'),
ek.TR_Field('TR.TotalAssetsReported'),
ek.TR_Field('TR.TotalLiabilities'),
ek.TR_Field('TR.TotalLongTermDebt'),
ek.TR_Field('TR.TotalEquity'),
ek.TR_Field('TR.EBIT'),
ek.TR_Field('TR.NetIncomeBeforeTaxes'),
ek.TR_Field('TR.OperatingIncome'),
ek.TR_Field('TR.OperatingExpenses'),
ek.TR_Field('TR.CostOfRevenueTotal'),
ek.TR_Field('TR.GrossMargin'),
ek.TR_Field('TR.OperatingMarginPercent'),
ek.TR_Field('TR.TotalDebtOutstanding'),
ek.TR_Field('TR.TotalInventory'),
ek.TR_Field('TR.CashAndSTInvestments'),
ek.TR_Field('TR.TotalReceivablesNet'),
ek.TR_Field('TR.TRBCEconomicSector'),
ek.TR_Field('TR.PropertyPlantEquipmentTotalNet'),
ek.TR_Field('TR.GoodwillNet'),
ek.TR_Field('TR.TangibleBVPS'),
ek.TR_Field('TR.PriceTargetMean'),
ek.TR_Field('TR.NumberOfAnalysts'),
ek.TR_Field('TR.QuickRatio'),
ek.TR_Field('TR.CurrentRatio')
]

numerical_fields.sort()

In [ ]:
# List of pertinant categorical fields
categorical_fields = [ek.TR_Field('TR.TRBCEconomicSector'),
ek.TR_Field('TR.CommonName'),
ek.TR_Field('TR.HeadquartersCountry'),
ek.TR_Field('TR.ExchangeName')
]

categorical_fields.sort()

## Getting Eikon Tickers
Tickers read from files don't have the proper Eikon extensions. This function tries to find the appropriate Eikon extension for each ticker.

In [ ]:
# Getting correct ticker name
def get_eikon_ticker_name(ticker,f = [ek.TR_Field('TR.RevenuePerShare')], end_date = '2019-01-01'):

    ticker = str(ticker)
    ticker1 = str(ticker)
    
    ticker = ticker1 + '.O'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date,
                                                                      'Frq': 'D'})
    if err is None:
        return ticker

    ticker = ticker1 + '.OQ'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date, 
                                                                      'Frq': 'D'})
    if err is None:
        return ticker
    
    ticker = ticker1 + '.TO'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date, 
                                                                      'Frq': 'D'})
    if err is None:
        return ticker

    ticker = ticker1 + '.N'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date, 
                                                                      'Frq': 'D'})
    if err is None:
        return ticker
    
    ticker = ticker1
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': '2018-01-01'})

    if err is None:
        return ticker

    print('Ticker {} not found'.format(ticker1))
    return None

## Grab Fundamental Data

In [ ]:
# Getting Data for Analysis

def get_fundamental_data(ticker, f,f1, start_date1, end_date1):
    
    try:
        # Grab numerical data for each quarter.
        df_fundamentals,err = ek.get_data([ticker], fields= f,parameters={'SDate': start_date1, 'EDate': end_date1,
                                                                'Period':'FQ0',
                                                               'Frq': 'FQ' })
        # Grab categorical data for each quarter.
        sector, err = ek.get_data([ticker], f1)
        
        df_fundamentals = df_fundamentals.merge(sector,how='left', on='Instrument')
        
        return(df_fundamentals)
    
    except:
        
        print('Error for {}'.format(ticker))
        return(0)

In [ ]:
# Getting Start and End Dates

start_date = dt.datetime(1990,1,1)
end_date = dt.datetime.today()
start_date1 = start_date.strftime('%Y-%m-%d')
end_date1 = end_date.strftime('%Y-%m-%d')

## Creating fundamentals dataset

In [ ]:
def write_fund_data(final_file, file_name):
    df_fundamentals = final_file
    df_fundamentals.sort_values(by= ['Instrument','Date'],inplace=True)
    
    # Handling missing dividend data
    df_fundamentals['Has Div'] = 0
    df_fundamentals.loc[df_fundamentals['Dividend Per Share - Mean'].isnull() == False, 'Has Div'] = 1
    
    # Fill nas with zero for dividends
    df_fundamentals['Dividend Per Share - Mean']= np.nan_to_num(df_fundamentals['Dividend Per Share - Mean'])

    # Write data to csv.
    df_fundamentals.to_csv(file_name)
    print("File Printed")

In [ ]:
final_file = None
cur_file = None
write_file = False
for i,tick in enumerate(ticker_list):
    
    # Write data to csv every 500 iterations.
    # Flag used in case on the 500th iteration the API call fails.
    # Want to make it so the current dataframe is written on the next successsful API call.
    if i > 0 and i % 500 == 0:
        write_file = True
    try:
        tick = get_eikon_ticker_name(tick, end_date = end_date1)
        if tick:
            cur_file = get_fundamental_data(tick, numerical_fields, categorical_fields,
                                            start_date1, end_date1)

        if (final_file is None and isinstance(cur_file, pd.DataFrame)):
            final_file = cur_file
        
        # Write to csv evey 500 tickers, if the final_file dataframe is populated.
        elif (write_file and isinstance(final_file, pd.DataFrame)):
            write_file = False
            file_name = '/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_2019_Fund_Data/NASDAQ_fund_data_{}.csv'.format(i)
            write_fund_data(final_file, file_name)
            if isinstance(cur_file, pd.DataFrame):
                final_file = cur_file
            else:
                final_file = None

        elif (isinstance(cur_file, pd.DataFrame)):
            final_file = final_file.append(cur_file, ignore_index = True)

    except urllib2.HTTPError as err:
        print('HTTPError for tick {}'.format(tick))
        print('Error message: {}'.format(err))
    except:
        print('Some error occurred for tick {}.'.format(tick))
    
   
    print(i)

if isinstance(final_file, pd.DataFrame): 
    file_name = '/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_2019_Fund_Data/NASDAQ_fund_data_{}.csv'.format(len(ticker_list))
    write_fund_data(final_file, file_name)
print('DONE!')

In [ ]:
final_file

## Cleaning up the dataset

In [ ]:
# Already done above now.

### Handling all remaining NA data ###
# Leaving this for later cleanup.
# df_fundamentals.dropna(inplace= True)